### Linear Regression Analysis of ADR
Using the transformed dataset, I converted the years into columns. Year(2015, 2016, 2017) becomes a dummy variable  

In [1]:
import pandas as pd
import numpy as np
import calendar
from datetime import datetime
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
hotel_data1 = pd.read_csv('hotel_data1.csv')
hotel_data1.head()

,hotel,arrival_date,overnight,adults,children,babies,meal,reserved_room,booking_changes,deposit_type,agent,company,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,length_of_stay
0,0,2015-07-01,0,2,0,0,1,1,3,0,0,0,0.0,0,0,Check-Out,2015-07-01,0
1,0,2015-07-01,0,2,0,0,1,1,4,0,0,0,0.0,0,0,Check-Out,2015-07-01,0
2,0,2015-07-01,1,1,0,0,1,0,0,0,0,0,75.0,0,0,Check-Out,2015-07-02,1
3,0,2015-07-01,1,1,0,0,1,1,0,0,1,0,75.0,0,0,Check-Out,2015-07-02,1
4,0,2015-07-01,1,2,0,0,1,1,0,0,1,0,98.0,0,1,Check-Out,2015-07-03,2


In [3]:
# check the data types of each column
hotel_data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75166 entries, 0 to 75165
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   hotel                        75166 non-null  int64  
 1   arrival_date                 75166 non-null  object 
 2   overnight                    75166 non-null  int64  
 3   adults                       75166 non-null  int64  
 4   children                     75166 non-null  int64  
 5   babies                       75166 non-null  int64  
 6   meal                         75166 non-null  int64  
 7   reserved_room                75166 non-null  int64  
 8   booking_changes              75166 non-null  int64  
 9   deposit_type                 75166 non-null  int64  
 10  agent                        75166 non-null  int64  
 11  company                      75166 non-null  int64  
 12  adr                          75166 non-null  float64
 13  required_car_par

In [4]:
# convert columns to the right data type
hotel_data1['arrival_date'] = pd.to_datetime(hotel_data1['arrival_date'])
hotel_data1['reservation_status_date'] = pd.to_datetime(hotel_data1['reservation_status_date'])

In [5]:
# create columns for years
year_data = hotel_data1.copy()
year_data['arrival_date'] = year_data['arrival_date'].dt.year
year_data['2015'] = np.where(year_data['arrival_date'] == 2015, 1, 0)
year_data['2016'] = np.where(year_data['arrival_date'] == 2016, 1, 0)
year_data['2017'] = np.where(year_data['arrival_date'] == 2017, 1, 0)
year_data.drop('arrival_date', axis=1, inplace=True)
year_data.head()

,hotel,overnight,adults,children,babies,meal,reserved_room,booking_changes,deposit_type,agent,company,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,length_of_stay,2015,2016,2017
0,0,0,2,0,0,1,1,3,0,0,0,0.0,0,0,Check-Out,2015-07-01,0,1,0,0
1,0,0,2,0,0,1,1,4,0,0,0,0.0,0,0,Check-Out,2015-07-01,0,1,0,0
2,0,1,1,0,0,1,0,0,0,0,0,75.0,0,0,Check-Out,2015-07-02,1,1,0,0
3,0,1,1,0,0,1,1,0,0,1,0,75.0,0,0,Check-Out,2015-07-02,1,1,0,0
4,0,1,2,0,0,1,1,0,0,1,0,98.0,0,1,Check-Out,2015-07-03,2,1,0,0


In [6]:
# check for correlation in the years 
year_data.corr()

,hotel,overnight,adults,children,babies,meal,reserved_room,booking_changes,deposit_type,agent,company,adr,required_car_parking_spaces,total_of_special_requests,length_of_stay,2015,2016,2017
hotel,1.000000,0.031824,-0.004021,-0.007925,-0.036705,0.137801,0.133498,-0.049784,-0.053104,0.151367,-0.056485,0.147908,-0.239655,0.040967,-0.229184,-0.059399,0.019970,0.027718
overnight,0.031824,1.000000,0.048233,0.007405,0.002557,0.001319,0.087538,-0.000289,0.005165,0.035552,-0.011384,0.194154,0.023416,0.024022,0.125747,-0.010026,-0.018268,0.027645
adults,-0.004021,0.048233,1.000000,0.043202,0.028784,0.008867,0.061113,-0.069437,-0.011950,0.242841,-0.302433,0.334175,0.038672,0.204176,0.149573,-0.041889,0.011655,0.022124
children,-0.007925,0.007405,0.043202,1.000000,0.025052,0.015282,0.014349,0.047858,-0.010366,0.040554,-0.062032,0.319919,0.075011,0.094490,0.017413,-0.033528,0.000317,0.027273
babies,-0.036705,0.002557,0.028784,0.025052,1.000000,-0.005285,-0.011614,0.079817,-0.004964,-0.004218,-0.018451,0.034152,0.034384,0.094595,0.018303,0.011610,-0.002689,-0.006707
meal,0.137801,0.001319,0.008867,0.015282,-0.005285,1.000000,0.003221,-0.022298,0.005894,0.044506,0.004162,0.014911,0.017790,0.067893,-0.042959,0.003741,0.004756,-0.008128
reserved_room,0.133498,0.087538,0.061113,0.014349,-0.011614,0.003221,1.000000,-0.055020,-0.003081,0.101476,-0.091192,0.161389,-0.035661,0.051675,0.119555,-0.096856,-0.026809,0.108209
booking_changes,-0.049784,-0.000289,-0.069437,0.047858,0.079817,-0.022298,-0.055020,1.000000,0.023385,-0.079427,0.078437,0.013968,0.042575,-0.003353,0.115663,-0.025224,-0.002699,0.023636
deposit_type,-0.053104,0.005165,-0.011950,-0.010366,-0.004964,0.005894,-0.003081,0.023385,1.000000,-0.070124,0.093641,-0.030271,-0.000608,-0.043033,-0.009629,-0.017420,0.044964,-0.033376
agent,0.151367,0.035552,0.242841,0.040554,-0.004218,0.044506,0.101476,-0.079427,-0.070124,1.000000,-0.614124,0.169269,-0.089837,0.152273,0.162946,-0.015031,-0.006234,0.018993


`adr` seems to have correlation with `adults`, `children` and `special requests`. It is not correlated with length of stay.

In [7]:
# qty_vars = ['overnight', 'adults', 'children', 'babies',
#                              'booking_changes','adr', 'required_car_parking_spaces', 'total_of_special_requests']
# fig = sns.PairGrid(data=hotel_data1, vars=qty_vars)
# fig.map(plt.scatter)
# plt.show()

In [11]:
year_data.shape[1]

20

In [8]:
year_data.columns

Index(['hotel', 'overnight', 'adults', 'children', 'babies', 'meal',
       'reserved_room', 'booking_changes', 'deposit_type', 'agent', 'company',
       'adr', 'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date', 'length_of_stay',
       '2015', '2016', '2017'],
      dtype='object')

In [9]:
# define independent and dependent variables. 
y = year_data['adr']
x1 = year_data.drop(['reservation_status','reservation_status_date','adr', '2017'], axis=1)
x = sm.add_constant(x1)

In [10]:
# model 1 - all quantitative data including binary data
results = sm.OLS(y,x).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    adr   R-squared:                       0.310
Model:                            OLS   Adj. R-squared:                  0.310
Method:                 Least Squares   F-statistic:                     2113.
Date:                Sun, 05 Feb 2023   Prob (F-statistic):               0.00
Time:                        23:40:23   Log-Likelihood:            -3.8554e+05
No. Observations:               75166   AIC:                         7.711e+05
Df Residuals:                   75149   BIC:                         7.713e+05
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                         -38.4666      2.187    -17.590      0.000     -42.753     -34.180
hotel                          14.5656      0.340     42.875      0.000      13.900      15.232
overnight                      82.8721      1.596     51.919      0.000      79.744      86.001
adults                         25.3012      0.315     80.357      0.000      24.684      25.918
children                       35.4326      0.385     91.956      0.000      34.677      36.188
babies                          4.5656      1.331      3.429      0.001       1.956       7.175
meal                           -8.7470      1.404     -6.231      0.000     -11.498      -5.996
reserved_room                  11.0135      0.395     27.859      0.000      10.239      11.788
booking_changes                 2.0250      0.207      9.777      0.000       1.619       2.431
deposit_type                   -2.4675      2.788     -0.885      0.376      -7.931       2.996
agent                           1.3464      0.527      2.556      0.011       0.314       2.379
company                       -13.1168      0.739    -17.746      0.000     -14.565     -11.668
required_car_parking_spaces    14.5275      0.514     28.267      0.000      13.520      15.535
total_of_special_requests       5.7961      0.187     30.958      0.000       5.429       6.163
length_of_stay                 -0.1947      0.063     -3.084      0.002      -0.318      -0.071
2015                          -14.6489      0.440    -33.262      0.000     -15.512     -13.786
2016                          -12.0934      0.338    -35.728      0.000     -12.757     -11.430
==============================================================================
Omnibus:                    14160.006   Durbin-Watson:                   0.918
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            39113.904
Skew:                           1.011   Prob(JB):                         0.00
Kurtosis:                       5.898   Cond. No.                         93.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

This model has more variables than the previous ones. The Rsquared value has also increased which means it might contain better explanatory variables.